# Train Naive Bayes Classifier - Optimized

**Author:** phamlucchuong  
**Date:** 2025-01-18  
**Dataset:** raw_data_bayes.csv (with disease_id)  

Training Naive Bayes với:
- Multiple model comparison
- Cross-validation
- Parallel processing
- Detailed evaluation

In [1]:
import pandas as pd
import json
import pickle
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support
)
from collections import Counter
import warnings
import time
from datetime import datetime

warnings.filterwarnings('ignore')

print("="*70)
print("🚀 NAIVE BAYES CLASSIFIER TRAINING")
print("="*70)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Author: phamlucchuong")
print("="*70)
print("\n✅ Libraries loaded")

🚀 NAIVE BAYES CLASSIFIER TRAINING
Start time: 2025-11-19 11:50:42
Author: phamlucchuong

✅ Libraries loaded


## 1. Load dữ liệu

In [2]:
print("\n" + "="*70)
print("📂 LOADING DATA")
print("="*70)

# Load binary data
df = pd.read_csv("../../data/processed/naive_bayes_data.csv")
print(f"✓ Loaded binary data: {df.shape}")

# Load symptom mapping
with open("../../data/processed/symptom_mapping.json", "r", encoding="utf-8") as f:
    symptom_mapping = json.load(f)

all_symptoms = symptom_mapping["all_symptoms"]
print(f"✓ Loaded symptom mapping: {len(all_symptoms)} symptoms")

# Load disease mapping
with open("../../data/processed/disease_mapping.json", "r", encoding="utf-8") as f:
    disease_mapping = json.load(f)

print(f"✓ Loaded disease mapping: {len(disease_mapping)} diseases")

print(f"\n📊 Dataset Info:")
print(f"   Samples: {len(df)}")
print(f"   Features: {len(all_symptoms)}")
print(f"   Classes: {df['disease'].nunique()}")
print(f"   Columns: {df.columns.tolist()[:5]} ... (+{len(df.columns)-5} more)")

# Display sample
print(f"\n📋 Sample data (first 3 rows, first 8 columns):")
print(df.iloc[:3, :8])


📂 LOADING DATA
✓ Loaded binary data: (2173, 627)
✓ Loaded symptom mapping: 625 symptoms
✓ Loaded disease mapping: 20 diseases

📊 Dataset Info:
   Samples: 2173
   Features: 625
   Classes: 21
   Columns: ['disease_id', 'disease', 'Ban_đỏ', 'Bị_bít_mũi', 'Bị_nhức_đầu'] ... (+622 more)

📋 Sample data (first 3 rows, first 8 columns):
  disease_id                disease  Ban_đỏ  Bị_bít_mũi  Bị_nhức_đầu  Bị_sốt  \
0       D001  Cảm lạnh thông thường       0           0            0       0   
1       D001  Cảm lạnh thông thường       0           0            0       0   
2       D001  Cảm lạnh thông thường       0           0            0       0   

   Bị_sổ_mũi  Chảy_mũi  
0          0         0  
1          0         0  
2          0         0  


## 2. Phân tích phân bố dữ liệu

In [3]:
print("\n" + "="*70)
print("📊 DATA ANALYSIS")
print("="*70)

# Phân bố bệnh
disease_counts = df['disease'].value_counts()

print(f"\n📈 Phân bố mẫu theo bệnh:")
print(f"\n{'Disease':<45} {'ID':<8} {'Samples':>8} {'%':>7}")
print("="*70)

# Tạo reverse mapping (disease_name -> disease_id)
reverse_disease_map = {v: k for k, v in disease_mapping.items()}

for disease, count in disease_counts.items():
    disease_id = reverse_disease_map.get(disease, "N/A")
    percentage = count / len(df) * 100
    print(f"{disease:<45} {disease_id:<8} {count:>8} {percentage:>6.1f}%")

print("\n📊 Thống kê:")
print(f"   Mean:   {disease_counts.mean():.1f} samples/disease")
print(f"   Median: {disease_counts.median():.1f} samples/disease")
print(f"   Min:    {disease_counts.min()} samples")
print(f"   Max:    {disease_counts.max()} samples")
print(f"   Std:    {disease_counts.std():.1f}")

# Phân tích triệu chứng
print(f"\n🔍 Phân tích triệu chứng:")
symptom_counts = df[all_symptoms].sum().sort_values(ascending=False)
print(f"   Top 10 triệu chứng phổ biến nhất:")
for i, (symptom, count) in enumerate(symptom_counts.head(10).items(), 1):
    print(f"   {i:2d}. {symptom:<40} {count:>3} lần ({count/len(df)*100:.1f}%)")


📊 DATA ANALYSIS

📈 Phân bố mẫu theo bệnh:

Disease                                       ID        Samples       %
Bệnh Tim mạch vành (Đau thắt ngực, Nhồi máu cơ tim) D014          158    7.3%
Bệnh Tăng huyết áp (Tim mạch)                 D013          135    6.2%
Viêm dạ dày ruột (Gastroenteritis)            D011          130    6.0%
Thủy đậu (Chickenpox)                         D012          118    5.4%
Bệnh Phổi tắc nghẽn mạn tính (COPD)           D017          117    5.4%
Viêm khớp mạn tính                            D020          115    5.3%
Đái tháo đường (Tiểu đường)                   D015          110    5.1%
Viêm loét dạ dày và Tá tràng                  D018          101    4.6%
Bệnh Lao phổi                                 D005          100    4.6%
Cảm lạnh thông thường                         D001          100    4.6%
Viêm phế quản cấp                             D004          100    4.6%
Viêm họng và Viêm Amidan cấp                  D003          100    4.6%
Cảm cúm       

## 3. Chuẩn bị dữ liệu cho training

In [4]:
print("\n" + "="*70)
print("🔧 DATA PREPARATION")
print("="*70)

# Tách features và labels
X = df[all_symptoms].values
y = df["disease"].values

print(f"\n✓ Features (X): {X.shape}")
print(f"✓ Labels (y): {y.shape}")

# Lọc bỏ classes có ít hơn 2 mẫu
min_samples = 2
disease_counts = Counter(y)
valid_classes = [d for d, count in disease_counts.items() if count >= min_samples]
removed_classes = [d for d, count in disease_counts.items() if count < min_samples]

if removed_classes:
    print(f"\n⚠️  Removing {len(removed_classes)} classes with < {min_samples} samples:")
    for disease in removed_classes:
        disease_id = reverse_disease_map.get(disease, "N/A")
        print(f"   - {disease} ({disease_id}): {disease_counts[disease]} samples")
    
    mask = np.isin(y, valid_classes)
    X = X[mask]
    y = y[mask]
    print(f"\n✓ After filtering: {len(X)} samples, {len(valid_classes)} classes")
else:
    print(f"\n✓ All classes have >= {min_samples} samples")

# Laplace smoothing
smoothing_value = 0.01
X = X + smoothing_value
print(f"\n✓ Applied Laplace smoothing (alpha={smoothing_value})")

print(f"\n📊 Final data shape:")
print(f"   X: {X.shape}")
print(f"   y: {y.shape}")
print(f"   Classes: {len(np.unique(y))}")


🔧 DATA PREPARATION

✓ Features (X): (2173, 625)
✓ Labels (y): (2173,)

⚠️  Removing 1 classes with < 2 samples:
   - Bệnh Phổi tắc_nghẽn_mạn_tính (COPD) (N/A): 1 samples

✓ After filtering: 2172 samples, 20 classes

✓ Applied Laplace smoothing (alpha=0.01)

📊 Final data shape:
   X: (2172, 625)
   y: (2172,)
   Classes: 20


## 4. Split train/test

In [5]:
print("\n" + "="*70)
print("✂️  TRAIN/TEST SPLIT")
print("="*70)

test_size = 0.2
random_state = 42

# Try stratified split
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_size, 
        random_state=random_state, 
        stratify=y
    )
    split_method = "Stratified"
    print(f"✓ Using stratified split")
except ValueError as e:
    print(f"⚠️  Stratified split failed: {e}")
    print(f"   → Using random split")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_size, 
        random_state=random_state
    )
    split_method = "Random"

print(f"\n📊 Split summary:")
print(f"   Method: {split_method}")
print(f"   Test size: {test_size*100:.0f}%")
print(f"   Random state: {random_state}")
print(f"\n   Train: {len(X_train):>4} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Test:  {len(X_test):>4} samples ({len(X_test)/len(X)*100:.1f}%)")

# Class distribution
train_classes = len(np.unique(y_train))
test_classes = len(np.unique(y_test))

print(f"\n📈 Class distribution:")
print(f"   Train: {train_classes} unique classes")
print(f"   Test:  {test_classes} unique classes")

if train_classes != test_classes:
    print(f"\n⚠️  Warning: Train and test have different number of classes!")
    missing_in_test = set(y_train) - set(y_test)
    if missing_in_test:
        print(f"   Classes missing in test set: {missing_in_test}")


✂️  TRAIN/TEST SPLIT
✓ Using stratified split

📊 Split summary:
   Method: Stratified
   Test size: 20%
   Random state: 42

   Train: 1737 samples (80.0%)
   Test:   435 samples (20.0%)

📈 Class distribution:
   Train: 20 unique classes
   Test:  20 unique classes


## 5. Training Multiple Models

In [6]:
print("\n" + "="*70)
print("🚀 MODEL TRAINING")
print("="*70)

# Define models
models = {
    "MultinomialNB (α=0.1)": MultinomialNB(alpha=0.1),
    "MultinomialNB (α=0.5)": MultinomialNB(alpha=0.5),
    "MultinomialNB (α=1.0)": MultinomialNB(alpha=1.0),
    "MultinomialNB (α=2.0)": MultinomialNB(alpha=2.0),
    "ComplementNB (α=0.5)": ComplementNB(alpha=0.5),
    "ComplementNB (α=1.0)": ComplementNB(alpha=1.0),
}

results = {}
start_time = time.time()

for name, model in models.items():
    print(f"\n{'─'*70}")
    print(f"Training: {name}")
    print(f"{'─'*70}")
    
    model_start = time.time()
    
    # Train
    model.fit(X_train, y_train)
    train_time = time.time() - model_start
    
    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Metrics
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    # Cross-validation (nếu có đủ samples)
    try:
        n_splits = min(3, len(np.unique(y_train)))
        if n_splits >= 2:
            cv_scores = cross_val_score(
                model, X_train, y_train,
                cv=n_splits,
                scoring='accuracy',
                n_jobs=-1
            )
            cv_mean = cv_scores.mean()
            cv_std = cv_scores.std()
        else:
            cv_mean = 0
            cv_std = 0
    except:
        cv_mean = 0
        cv_std = 0
    
    # Precision, Recall, F1
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred_test, average='weighted', zero_division=0
    )
    
    results[name] = {
        'model': model,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'cv_mean': cv_mean,
        'cv_std': cv_std,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'train_time': train_time,
        'y_pred': y_pred_test
    }
    
    print(f"✓ Train Accuracy:  {train_acc:.4f}")
    print(f"✓ Test Accuracy:   {test_acc:.4f}")
    if cv_mean > 0:
        print(f"✓ CV Accuracy:     {cv_mean:.4f} (±{cv_std:.4f})")
    print(f"✓ Precision:       {precision:.4f}")
    print(f"✓ Recall:          {recall:.4f}")
    print(f"✓ F1-Score:        {f1:.4f}")
    print(f"⏱️  Training time:   {train_time:.4f}s")

total_time = time.time() - start_time
print(f"\n{'='*70}")
print(f"✅ All models trained in {total_time:.2f}s")
print(f"{'='*70}")


🚀 MODEL TRAINING

──────────────────────────────────────────────────────────────────────
Training: MultinomialNB (α=0.1)
──────────────────────────────────────────────────────────────────────
✓ Train Accuracy:  0.9942
✓ Test Accuracy:   0.9862
✓ CV Accuracy:     0.9856 (±0.0059)
✓ Precision:       0.9870
✓ Recall:          0.9862
✓ F1-Score:        0.9863
⏱️  Training time:   0.0185s

──────────────────────────────────────────────────────────────────────
Training: MultinomialNB (α=0.5)
──────────────────────────────────────────────────────────────────────
✓ Train Accuracy:  0.9919
✓ Test Accuracy:   0.9839
✓ CV Accuracy:     0.9799 (±0.0043)
✓ Precision:       0.9848
✓ Recall:          0.9839
✓ F1-Score:        0.9839
⏱️  Training time:   0.0098s

──────────────────────────────────────────────────────────────────────
Training: MultinomialNB (α=1.0)
──────────────────────────────────────────────────────────────────────
✓ Train Accuracy:  0.9908
✓ Test Accuracy:   0.9816
✓ CV Accuracy: 

## 6. Model Comparison

In [7]:
print("\n" + "="*70)
print("📊 MODEL COMPARISON")
print("="*70)

# Create comparison table
comparison_data = []
for name, res in results.items():
    comparison_data.append({
        'Model': name,
        'Train Acc': f"{res['train_acc']:.4f}",
        'Test Acc': f"{res['test_acc']:.4f}",
        'CV Acc': f"{res['cv_mean']:.4f}" if res['cv_mean'] > 0 else "N/A",
        'Precision': f"{res['precision']:.4f}",
        'Recall': f"{res['recall']:.4f}",
        'F1': f"{res['f1']:.4f}",
        'Time (s)': f"{res['train_time']:.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

# Select best model
best_model_name = max(results, key=lambda x: results[x]['test_acc'])
best_result = results[best_model_name]
best_model = best_result['model']

print(f"\n{'='*70}")
print(f"🏆 BEST MODEL: {best_model_name}")
print(f"{'='*70}")
print(f"   Test Accuracy:  {best_result['test_acc']:.4f}")
print(f"   Precision:      {best_result['precision']:.4f}")
print(f"   Recall:         {best_result['recall']:.4f}")
print(f"   F1-Score:       {best_result['f1']:.4f}")
print(f"   Training time:  {best_result['train_time']:.4f}s")


📊 MODEL COMPARISON

                Model Train Acc Test Acc CV Acc Precision Recall     F1 Time (s)
MultinomialNB (α=0.1)    0.9942   0.9862 0.9856    0.9870 0.9862 0.9863   0.0185
MultinomialNB (α=0.5)    0.9919   0.9839 0.9799    0.9848 0.9839 0.9839   0.0098
MultinomialNB (α=1.0)    0.9908   0.9816 0.9729    0.9828 0.9816 0.9815   0.0093
MultinomialNB (α=2.0)    0.9856   0.9770 0.9603    0.9791 0.9770 0.9768   0.0070
 ComplementNB (α=0.5)    0.9620   0.9563 0.9326    0.9607 0.9563 0.9555   0.0077
 ComplementNB (α=1.0)    0.9620   0.9563 0.9315    0.9607 0.9563 0.9555   0.0078

🏆 BEST MODEL: MultinomialNB (α=0.1)
   Test Accuracy:  0.9862
   Precision:      0.9870
   Recall:         0.9862
   F1-Score:       0.9863
   Training time:  0.0185s


## 7. Detailed Evaluation

In [8]:
print("\n" + "="*70)
print(f"📊 DETAILED EVALUATION - {best_model_name}")
print("="*70)

y_pred_best = best_result['y_pred']

# Classification Report
print("\n📋 Classification Report:")
print("\n" + classification_report(y_test, y_pred_best, zero_division=0))

# Per-class accuracy
print("\n📈 Per-class Performance:")
print(f"\n{'Disease':<45} {'ID':<8} {'Acc':>6} {'Samples':>8}")
print("="*70)

for disease in sorted(np.unique(y_test)):
    mask = y_test == disease
    if mask.sum() > 0:
        acc = accuracy_score(y_test[mask], y_pred_best[mask])
        disease_id = reverse_disease_map.get(disease, "N/A")
        n_samples = mask.sum()
        print(f"{disease:<45} {disease_id:<8} {acc:>5.1%} {n_samples:>8}")

# Confusion Matrix (top 10 classes)
print("\n📊 Confusion Matrix (Top 10 most common classes in test set):")
unique_test_classes = np.unique(y_test)

if len(unique_test_classes) > 10:
    test_counts = Counter(y_test)
    top_classes = [c for c, _ in test_counts.most_common(10)]
    
    mask = np.isin(y_test, top_classes)
    y_test_top = y_test[mask]
    y_pred_top = y_pred_best[mask]
    
    cm = confusion_matrix(y_test_top, y_pred_top, labels=top_classes)
    cm_df = pd.DataFrame(cm, index=top_classes, columns=top_classes)
else:
    cm = confusion_matrix(y_test, y_pred_best)
    cm_df = pd.DataFrame(cm, index=unique_test_classes, columns=unique_test_classes)

print("\n" + cm_df.to_string())


📊 DETAILED EVALUATION - MultinomialNB (α=0.1)

📋 Classification Report:

                                                     precision    recall  f1-score   support

                                    Bệnh Gút (Gout)       1.00      1.00      1.00        18
                                      Bệnh Lao phổi       0.95      0.95      0.95        20
                Bệnh Phổi tắc nghẽn mạn tính (COPD)       1.00      1.00      1.00        24
                            Bệnh Tay - Chân - Miệng       1.00      1.00      1.00        20
Bệnh Tim mạch vành (Đau thắt ngực, Nhồi máu cơ tim)       1.00      0.94      0.97        32
                                 Bệnh Tiêu chảy cấp       1.00      1.00      1.00        20
                      Bệnh Tăng huyết áp (Tim mạch)       0.90      1.00      0.95        27
                                            Cảm cúm       1.00      1.00      1.00        20
                              Cảm lạnh thông thường       1.00      1.00      1.00      

## 8. Save Model

In [9]:
import os

print("\n" + "="*70)
print("💾 SAVING MODEL")
print("="*70)

os.makedirs("../../models", exist_ok=True)

# 1. Save best model
model_path = "../../models/naive_bayes_model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(best_model, f)
print(f"\n✓ Saved model: {model_path}")

# 2. Save model info
model_info = {
    "model_name": best_model_name,
    "model_type": type(best_model).__name__,
    "symptoms": all_symptoms,
    "diseases": sorted(np.unique(y_train).tolist()),
    "disease_mapping": disease_mapping,
    "metrics": {
        "test_accuracy": float(best_result['test_acc']),
        "precision": float(best_result['precision']),
        "recall": float(best_result['recall']),
        "f1_score": float(best_result['f1']),
        "cv_accuracy_mean": float(best_result['cv_mean']) if best_result['cv_mean'] > 0 else None,
        "cv_accuracy_std": float(best_result['cv_std']) if best_result['cv_std'] > 0 else None
    },
    "data_info": {
        "n_samples_train": len(X_train),
        "n_samples_test": len(X_test),
        "n_features": len(all_symptoms),
        "n_classes": len(np.unique(y_train)),
        "split_method": split_method,
        "test_size": test_size,
        "random_state": random_state,
        "smoothing_value": smoothing_value
    },
    "training_info": {
        "training_time": float(best_result['train_time']),
        "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "author": "phamlucchuong"
    }
}

info_path = "../../models/model_info.json"
with open(info_path, "w", encoding="utf-8") as f:
    json.dump(model_info, f, ensure_ascii=False, indent=2)
print(f"✓ Saved info: {info_path}")

# 3. Save all model results for comparison
all_results_path = "../../models/all_models_comparison.json"
all_results_data = {
    name: {
        "train_accuracy": float(res['train_acc']),
        "test_accuracy": float(res['test_acc']),
        "cv_mean": float(res['cv_mean']) if res['cv_mean'] > 0 else None,
        "cv_std": float(res['cv_std']) if res['cv_std'] > 0 else None,
        "precision": float(res['precision']),
        "recall": float(res['recall']),
        "f1_score": float(res['f1']),
        "training_time": float(res['train_time'])
    }
    for name, res in results.items()
}

with open(all_results_path, "w", encoding="utf-8") as f:
    json.dump(all_results_data, f, ensure_ascii=False, indent=2)
print(f"✓ Saved comparison: {all_results_path}")

print(f"\n📦 Saved files:")
print(f"   1. {model_path}")
print(f"   2. {info_path}")
print(f"   3. {all_results_path}")


💾 SAVING MODEL

✓ Saved model: ../../models/naive_bayes_model.pkl
✓ Saved info: ../../models/model_info.json
✓ Saved comparison: ../../models/all_models_comparison.json

📦 Saved files:
   1. ../../models/naive_bayes_model.pkl
   2. ../../models/model_info.json
   3. ../../models/all_models_comparison.json


## 9. Test với dữ liệu mới

In [10]:
print("\n" + "="*70)
print("🧪 TESTING WITH NEW DATA")
print("="*70)

def predict_disease(symptoms_list, model, all_symptoms, disease_mapping, top_k=5):
    """
    Dự đoán bệnh từ danh sách triệu chứng
    
    Args:
        symptoms_list: List of symptoms
        model: Trained model
        all_symptoms: List of all possible symptoms
        disease_mapping: Dict mapping disease_id to disease_name
        top_k: Number of top predictions
    
    Returns:
        prediction, top_diseases, disease_id
    """
    # Convert to binary vector
    vector = np.array([1 if s in symptoms_list else 0 for s in all_symptoms])
    vector = vector.reshape(1, -1) + smoothing_value
    
    # Predict
    prediction = model.predict(vector)[0]
    probabilities = model.predict_proba(vector)[0]
    
    # Get disease_id
    reverse_map = {v: k for k, v in disease_mapping.items()}
    disease_id = reverse_map.get(prediction, "Unknown")
    
    # Top K
    top_indices = np.argsort(probabilities)[::-1][:top_k]
    top_diseases = []
    for i in top_indices:
        disease_name = model.classes_[i]
        disease_id_top = reverse_map.get(disease_name, "Unknown")
        top_diseases.append((disease_name, disease_id_top, probabilities[i]))
    
    return prediction, top_diseases, disease_id

# Test cases
test_cases = [
    {
        "name": "Case 1: Cúm",
        "symptoms": ["sốt_cao_trên_38_5_độ", "đau_đầu_dữ_dội", "đau_nhức_cơ_bắp", "ho_khan"],
        "expected": "Cảm cúm (Influenza)"
    },
    {
        "name": "Case 2: Cảm lạnh",
        "symptoms": ["hắt_hơi", "sổ_mũi", "nghẹt_mũi", "đau_họng_nhẹ"],
        "expected": "Cảm lạnh thông thường"
    },
    {
        "name": "Case 3: Viêm họng",
        "symptoms": ["đau_họng_dữ_dội", "khó_nuốt", "sốt", "amidan_sưng_đỏ"],
        "expected": "Viêm họng và Viêm Amidan cấp"
    },
    {
        "name": "Case 4: Tiêu chảy",
        "symptoms": ["tiêu_chảy_phân_lỏng", "đau_bụng_quặn", "sốt_nhẹ", "mất_nước"],
        "expected": "Bệnh Tiêu chảy cấp"
    },
    {
        "name": "Case 5: Gút",
        "symptoms": ["đau_khớp_ngón_chân_cái_dữ_dội", "sưng_đỏ_nóng", "đau_đột_ngột_ban_đêm"],
        "expected": "Bệnh Gút (Gout)"
    },
]

print("\n📝 Test Results:\n")

correct = 0
total = len(test_cases)

for i, test in enumerate(test_cases, 1):
    symptoms = test["symptoms"]
    expected = test.get("expected", "Unknown")
    
    pred, top_diseases, disease_id = predict_disease(
        symptoms, best_model, all_symptoms, disease_mapping
    )
    
    is_correct = pred == expected
    if is_correct:
        correct += 1
    
    print(f"{test['name']}:")
    print(f"   Symptoms: {', '.join(symptoms[:3])}" + (" ..." if len(symptoms) > 3 else ""))
    print(f"   Expected: {expected}")
    print(f"   Predicted: {pred} ({disease_id}) {'✅' if is_correct else '❌'}")
    print(f"   \n   Top 5 predictions:")
    for rank, (disease, did, prob) in enumerate(top_diseases, 1):
        marker = "👉" if disease == pred else "  "
        print(f"   {marker} {rank}. {disease:<45} ({did:<8}) {prob:>6.2%}")
    print()

print(f"{'='*70}")
print(f"📊 Test Accuracy: {correct}/{total} ({correct/total*100:.1f}%)")
print(f"{'='*70}")


🧪 TESTING WITH NEW DATA

📝 Test Results:

Case 1: Cúm:
   Symptoms: sốt_cao_trên_38_5_độ, đau_đầu_dữ_dội, đau_nhức_cơ_bắp ...
   Expected: Cảm cúm (Influenza)
   Predicted: Cảm cúm (D002) ❌
   
   Top 5 predictions:
   👉 1. Cảm cúm                                       (D002    ) 88.90%
      2. Sốt xuất huyết                                (D006    )  2.17%
      3. Viêm họng và Viêm Amidan cấp                  (D003    )  1.70%
      4. Sốt rét                                       (D007    )  0.99%
      5. Viêm phế quản cấp                             (D004    )  0.97%

Case 2: Cảm lạnh:
   Symptoms: hắt_hơi, sổ_mũi, nghẹt_mũi ...
   Expected: Cảm lạnh thông thường
   Predicted: Cảm cúm (D002) ❌
   
   Top 5 predictions:
   👉 1. Cảm cúm                                       (D002    ) 44.23%
      2. Viêm họng và Viêm Amidan cấp                  (D003    ) 11.02%
      3. Viêm dạ dày ruột (Gastroenteritis)            (D011    )  7.13%
      4. Cảm lạnh thông thường                

## 10. Summary

In [11]:
print("\n" + "="*70)
print("✅ TRAINING SUMMARY")
print("="*70)

print(f"\n📊 Dataset:")
print(f"   Total samples: {len(df)}")
print(f"   Train samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Features: {len(all_symptoms)}")
print(f"   Classes: {len(np.unique(y_train))}")

print(f"\n🏆 Best Model:")
print(f"   Name: {best_model_name}")
print(f"   Test Accuracy: {best_result['test_acc']:.2%}")
print(f"   F1-Score: {best_result['f1']:.2%}")

print(f"\n💾 Saved Files:")
print(f"   - naive_bayes_model.pkl")
print(f"   - model_info.json")
print(f"   - all_models_comparison.json")

print(f"\n⏱️  Training Info:")
print(f"   Training time: {best_result['train_time']:.4f}s")
print(f"   Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"   Author: phamlucchuong")

print(f"\n" + "="*70)
print("✅ DONE!")
print("="*70)

print(f"\n📝 Next steps:")
print(f"   1. Review model performance in model_info.json")
print(f"   2. Test model with API: python api/main.py")
print(f"   3. Deploy to production")


✅ TRAINING SUMMARY

📊 Dataset:
   Total samples: 2173
   Train samples: 1737
   Test samples: 435
   Features: 625
   Classes: 20

🏆 Best Model:
   Name: MultinomialNB (α=0.1)
   Test Accuracy: 98.62%
   F1-Score: 98.63%

💾 Saved Files:
   - naive_bayes_model.pkl
   - model_info.json
   - all_models_comparison.json

⏱️  Training Info:
   Training time: 0.0185s
   Timestamp: 2025-11-19 11:50:49
   Author: phamlucchuong

✅ DONE!

📝 Next steps:
   1. Review model performance in model_info.json
   2. Test model with API: python api/main.py
   3. Deploy to production
